<a href="https://colab.research.google.com/github/govindmurthi21/statlearning/blob/main/CSXPDFReader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain-community langchain-openai
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

In [2]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from google.colab import userdata

In [3]:
import os
os.environ["OPEN_API_KEY"] = userdata.get("openapi_key")

In [4]:
pdfReader = PdfReader('/content/sample_data/CSXI_301.pdf')

In [5]:
from typing_extensions import Concatenate
raw_text = ''
for i, page in enumerate(pdfReader.pages):
  content = page.extract_text()
  if content:
    raw_text += content

In [6]:
raw_text

' \n     \n \n \n         \nCSX Intermodal \nEDI 301  \nPrivate CY Bookings \n \nCSX Intermodal EDI 301 Implementation Guide for Customers \n \nX12/V4010/301: 301 Confirmation (Ocean)  \n \nVersion: 004010 \n \nQuestions about the CSXI EDI 301 Implem entation Guide should be directed to: \n877-SHIPCSX option 2, prompt 2 \nMonday – Friday: 7:00 AM – 6:00 PM 9/24/2008 Confirmation (Ocean) - 301\n \nCSXI_301_004010.ecs i For internal use only\n  \nTable of Contents  \n \n301  Confirmation (Ocean) ..................................................................................................... 1 \nISA  Interchange Control Header .......................................................................................... 2 \nGS  Functional Group Header ............................................................................................. 4 \nST  Transaction Set Header ................................................................................................ 6 \nB1  Beginning 

In [7]:
text_splitter = CharacterTextSplitter(
    separator= '\n',
    chunk_size=800,
    chunk_overlap=200,
    length_function=len
)
texts = text_splitter.split_text(raw_text)

In [8]:
len(texts)

46

In [9]:
embeddings = OpenAIEmbeddings(openai_api_key=os.environ["OPEN_API_KEY"])

<ipython-input-9-3901eddd41b2>:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=os.environ["OPEN_API_KEY"])


In [10]:
document_search = FAISS.from_texts(texts, embeddings)

In [14]:
from langchain.chains.question_answering import load_qa_chain
from langchain_openai import OpenAI

In [16]:
chain = load_qa_chain(OpenAI(openai_api_key=os.environ["OPEN_API_KEY"]), chain_type="stuff")

<ipython-input-16-c7e9059d2c45>:1: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(OpenAI(openai_api_key=os.environ["OPEN_API_KEY"]), chain_type="stuff")


In [18]:
query = "What is ISA02?"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

' ISA02 is the Authorization Information element, which is used for additional identification or authorization of the interchange sender or data in the interchange. Its value is determined by the Authorization Information Qualifier (I01).'